# Ajuste hyperparametros RF

In [1]:
from sklearn.linear_model import Ridge
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
import pandas as pd
import numpy as np
import math
import os
import json
import joblib

from funcionesComunes import *

# Bucle de ajuste de hiperparametros

In [ ]:
for archivo in os.listdir("../Datasets/RCPMerged/"):
    print(f"Procesando archivo: {os.path.splitext(archivo)[0]}")

    # Leemos el archivo de pindrow y eleiminamos la poblacion INDI005
    df = pd.read_csv(f"../Datasets/RCPMerged/{archivo}")
    df = df[~df["nametag"].str.startswith("INDI005")]

    df = codification(df)

    # Ejecutamos el cambio de la estructura de datos para el modelo incluyendo el clima simulado
    df = dataStructureSimulatedClimate(df)

    # Normalizamos los datos de crecimiento de los individuos
    df, valorNormalizacion = individualNormalization(df)

    train_data, val_data, test_data = split_population_individuals(df, train_pct=0.80, val_pct_in_train=0.20, details=False)

    X_train = train_data.drop("bai", axis="columns")
    y_train = train_data["bai"]
    X_val = val_data.drop("bai", axis="columns")
    y_val = val_data["bai"]
    X_test = test_data.drop("bai", axis="columns")
    y_test = test_data["bai"]
    X_train.shape, y_train.shape, X_test.shape, y_test.shape

    # Combina los conjuntos de entrenamiento y validación
    X_combined = np.vstack((X_train, X_val))
    y_combined = np.hstack((y_train, y_val))

    # Define los índices para entrenamiento con -1 y para validación con 0
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_val.shape[0])]

    # Crea el objeto PredefinedSplit
    ps = PredefinedSplit(test_fold)

    data_to_save = {}

    reg = Ridge()
    
    # Parametros de grid search
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1, 10],  # Valores de alpha más variados
        'max_iter': [100, 200, 300, 400, 500],  # Más iteraciones
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'saga'],  # Diferentes solvers
        'tol': [ 1e-3, 1e-2, 1e-1]  # Tolerancia de la convergencia
    }

    # Configuración de GridSearchCV
    grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, cv=ps, scoring='neg_root_mean_squared_error', verbose=3)

    # Ajuste de GridSearchCV a tus datos
    grid_search.fit(X_combined, y_combined)

    # Mejores hiperparámetros encontrados
    print("Mejores hiperparámetros:", grid_search.best_params_)

    # Guardamos los resultados de los numberBestModels mejores modelos
    top_n_results = []

    # Obtener los índices de los 10 mejores modelos
    top_n_indices = np.argsort(grid_search.cv_results_['rank_test_score'])[:10]

    # Iterar sobre cada conjunto de hiperparámetros
    for idx in top_n_indices:
        
        # Extraemos los valores reales de los parámetros (no listas)
        params = grid_search.cv_results_['params'][idx]
        
        # Ajustamos el modelo con los parametros actuales
        model = Ridge(random_state=42, **params)
        model.fit(X_train, y_train)
        
        # Predecir en el conjunto de entrenamiento
        y_train_pred = model.predict(X_train)
        y_train_true_denorm = y_train.values * np.array([valorNormalizacion[nt] for nt in X_train["nametag_encoded"].values])
        y_train_pred_denorm = y_train_pred * np.array([valorNormalizacion[nt] for nt in X_train["nametag_encoded"].values])

        # Calcular métricas en el conjunto de entrenamiento
        train_mse = mean_squared_error(y_train_true_denorm, y_train_pred_denorm)
        train_rmse = np.sqrt(train_mse)
        train_r2 = r2_score(y_train_true_denorm, y_train_pred_denorm)
        train_mape = mean_absolute_percentage_error(y_train_true_denorm, y_train_pred_denorm) * 100
        
        # Predecir en el conjunto de validación
        y_val_pred = model.predict(X_val)
        y_val_true_denorm = y_val.values * np.array([valorNormalizacion[nt] for nt in X_val["nametag_encoded"].values])
        y_val_pred_denorm = y_val_pred * np.array([valorNormalizacion[nt] for nt in X_val["nametag_encoded"].values])
        

        # Calcular métricas en el conjunto de validación
        val_mse = mean_squared_error(y_val_true_denorm, y_val_pred_denorm)
        val_rmse = np.sqrt(val_mse)
        val_r2 = r2_score(y_val_true_denorm, y_val_pred_denorm)
        val_mape = mean_absolute_percentage_error(y_val_true_denorm, y_val_pred_denorm) * 100
        
        # Predecir en el conjunto de prueba
        y_test_pred = model.predict(X_test)
        y_test_true_denorm = y_test.values * np.array([valorNormalizacion[nt] for nt in X_test["nametag_encoded"].values])
        y_test_pred_denorm = y_test_pred * np.array([valorNormalizacion[nt] for nt in X_test["nametag_encoded"].values])

        # Calcular métricas en el conjunto de prueba
        test_mse = mean_squared_error(y_test_true_denorm, y_test_pred_denorm)
        test_rmse = np.sqrt(test_mse)
        test_r2 = r2_score(y_test_true_denorm, y_test_pred_denorm)
        test_mape = mean_absolute_percentage_error(y_test_true_denorm, y_test_pred_denorm) * 100
        
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

        # Crear un diccionario con toda la información
        model_info = {
            'params': params,
            'MSE_train': train_mse,
            'RMSE_train': train_rmse,
            'R2_train': train_r2,
            'MAPE_train': train_mape,
            'MSE_val': val_mse,
            'RMSE_val': val_rmse,
            'R2_val': val_r2,
            'MAPE_val': val_mape,
            'MSE_test': test_mse,
            'RMSE_test': test_rmse,
            'R2_test': test_r2,
            'MAPE_test': test_mape
        }
        
        # Agregar al listado de resultados
        top_n_results.append(model_info)

    # 3. Ordenar los modelos por RMSE de validación
    results_list_sorted = sorted(top_n_results, key=lambda x: x['RMSE_val'])

    # 4. Obtener los 10 mejores modelos y guardar en un archivo JSON
    top_10_models = results_list_sorted[:10]

    with open(f"resultados/LR/{os.path.splitext(archivo)[0]}_best_models_updated_no_train.json", 'w') as f:
        json.dump(top_10_models, f, indent=4)


    # Si deseas guardar los 10 mejores modelos
    for idx, model_info in enumerate(top_10_models):
        params = model_info['params']
        model = Ridge(**params)
        model.fit(X_train, y_train)
        filename = f'resultados/LR/{os.path.splitext(archivo)[0]}_modelo_top_{idx+1}.pkl'
        joblib.dump(model, filename)

